In [6]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from random import *
from matplotlib import pyplot as plt
import numpy as np
import GPy
from datetime import datetime

# change here
model_class_names = [3,6]

# randomly sample demo_light_num rows to model our function
sample_percentage = 1;

# sample sample_time_index_num time-indexes/columns from the data (1024 columns totally)
sample_time_index_num = 200;

m_train_set_path = "./MALLAT/MALLAT_TRAIN"

In [7]:
# sample_time_index_num time indexes from 1024 indexes
times_indexes = np.asarray(sorted(sample(list(range(0, 1024)), sample_time_index_num)))
# times_indexes = np.arange(0,1024, 1024/sample_time_index_num, dtype=int);
print("We choose the following ",sample_time_index_num, "time indexes from 1024 ones\n", times_indexes)

We choose the following  200 time indexes from 1024 ones
 [   3    5   14   17   31   59   63   76   77   83   85   93   94   97  100
  102  114  118  119  121  132  136  141  155  163  165  166  169  177  185
  191  192  193  196  198  201  202  204  208  209  218  220  221  225  235
  238  239  255  263  272  274  279  281  282  287  292  294  303  315  319
  321  325  331  336  343  347  348  350  355  356  358  362  365  366  369
  380  390  391  403  404  410  412  414  416  420  429  435  436  444  451
  452  455  458  464  465  474  476  481  491  495  497  498  507  516  521
  523  530  531  535  536  546  547  552  571  582  586  596  606  616  618
  624  633  639  640  644  649  651  657  659  665  671  674  678  682  687
  689  695  698  699  702  706  707  709  716  722  735  737  738  747  750
  751  760  762  779  781  789  791  801  805  814  818  828  830  831  832
  837  840  841  845  867  873  875  890  891  895  896  897  899  905  906
  915  920  924  933  936  950

In [8]:
# class name {1,2,3}
class_names = np.loadtxt(m_train_set_path, delimiter=',', usecols=[0])
m_data = np.loadtxt(m_train_set_path, delimiter=',',usecols=range(1,1025))

# normalize magnitude
m_data -= m_data.mean(1)[:,np.newaxis]
m_data /= m_data.std(1)[:,np.newaxis]

In [9]:
indices = [i for i,cn in enumerate(class_names) if cn in model_class_names]
print("model_class_names=",model_class_names," totally has ", len(indices), " rows");

demo_num = int(sample_percentage * len(indices))
indices = sorted(sample(indices, demo_num))
# indices = indices[0:demo_light_num]
print("We select the following ",len(indices)," rows\n",indices)

model_class_names= [3, 6]  totally has  12  rows
We select the following  12  rows
 [0, 8, 11, 12, 17, 30, 31, 38, 42, 44, 50, 54]


In [10]:
kern_class = GPy.kern.Matern32(input_dim=1, variance=1.5, lengthscale=2.5, active_dims=[0], name='class')
kern_replicate = GPy.kern.Matern32(input_dim=1, variance=.1, lengthscale=2.5, active_dims=[0], name='replicate')
k_hierarchy = GPy.kern.Hierarchical(kernels=[kern_class, kern_replicate])
print(k_hierarchy)

time_index_stack = np.tile(times_indexes.reshape(-1,1),(demo_num, 1)) ;

replicate_stack = []
for r in range(1, demo_num + 1):
    replicate_stack.append(np.ones(sample_time_index_num) * r);

replicate_stack = np.asarray(replicate_stack).reshape(-1,1);

filtered_data = m_data[indices]
filtered_data = filtered_data[:,times_indexes]

X = np.hstack((time_index_stack, replicate_stack))
X = np.vstack(X)
Y = np.vstack(filtered_data.reshape(-1,1))

print(X.shape)
print(Y.shape)

X_file_path = "./sample_size_train_code/mallat_model_files/X_" + str(model_class_names) + str(sample_percentage) + ".npy"
Y_file_path = "./sample_size_train_code/mallat_model_files/Y_" + str(model_class_names) + str(sample_percentage) + ".npy"

np.save(X_file_path, X)
np.save(Y_file_path, Y)

m = GPy.models.GPRegression(X=X, Y=Y, kernel=k_hierarchy)
m.optimize_restarts(num_restarts=2);
m.optimize('bfgs', messages=1)
print(m)

save_file_path = "./sample_size_train_code/mallat_model_files/model_save" + str(model_class_names) + str(sample_percentage)  + ".npy"
np.save(save_file_path, m.param_array)

  hierarchy.             |  value  |  constraints  |  priors
  class.variance         |    1.5  |      +ve      |        
  class.lengthscale      |    2.5  |      +ve      |        
  replicate.variance     |    0.1  |      +ve      |        
  replicate.lengthscale  |    2.5  |      +ve      |        
(2400, 2)
(2400, 1)
Optimization restart 1/2, f = -3792.7070369855874
Optimization restart 2/2, f = -3957.709523413446
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    06s55  0004  -3.980104e+03   3.136444e+02 
    21s94  0013  -3.969630e+03   6.532566e+04 
 01m03s79  0038  -3.991139e+03   1.169302e+02 
 03m01s49  0108  -3.999670e+03   9.207111e+02 
 03m30s01  0125  -3.999818e+03   6.638237e+02 
 03m45s06  0134  -4.000428e+03   9.641856e+00 
 04m23s72  0157  -4.001631e+03   1.263596e+02 
 04m32s12  0162  -4.001631e+03   1.295739e+02 
 04m50s43  0173  -4.001631e+03   1.263598e+02 
Runtime:  04m50s43
Optimization status: Converged


Name : GP